## 深度Q网络

表格型方法用表格的形式存储价值函数V(s)或动作价值函数Q(s,a)，但这样的方法局限性很大。使用Q网络近似状态价值函数可以使用一个函数直接拟合状态价值函数，省去了表格存储和检索的步骤。而且还能够对**连续**的状态空间建模。DQN的主要改动有三点：
- 使用深度神经网络替代Q表格；
- 使用了经验回放（Replay Buffer）：使用历史数据训练，比使用一次就扔掉大大提高了样本使用效率。还可以**减少样本之间的相关性**，原则上获取经验阶段和学习阶段是分开的，原来时序的训练数据有可能是不稳定的，打乱之后再学习有助于提高训练的稳定性，和深度学习中划分数据集时打乱样本是一个道理。
- 使用了两个网络：**策略网络**和**目标网络**，策略网络用来预测，目标网络用来计算目标值。每隔若干步才把每步更新的策略网络参数复制给目标网络，这样做也是为了稳定训练，避免Q值的发散。



### 程序实现

#### 调用相关的包

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import math
import random
import gym
import seaborn as sns
import os
import argparse


#### 定义模型
使用一个三层MLP来构建模型。

In [79]:
class QNet(nn.Module):
    def __init__(self, n_states, n_actions, hidden_dim=256):
        '''初始化Q网络为全连接网络
        '''
        super(QNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_actions)
        )

    def forward(self, x):
        return self.net(x)

#### 定义经验回放
经验回放是有一定容量的，只有存储一定的transition网络才会更新，否则就退回了之前的逐步更新。经验回放需要包含两个功能或方法：
- push：将新的transition添加到经验回放中，满了就把最开始放进去的样本挤掉，因此推荐用队列来写；
- sample：随机采样出一个或者若干样本提供给DQN。

In [80]:
class ReplayBuffer(object):
    def __init__(self, capacity: int) -> None:
        self.capacity = capacity
        self.buffer = deque(maxlen=self.capacity)
    
    def push(self, transitions):
        self.buffer.append(transitions)
    
    def sample(self, batch_size: int, sequential: bool = False):
        if batch_size > len(self.buffer): # 如果批量大小大于经验回放的容量 则取全部经验回放容量
            batch_size = len(self.buffer)
        if sequential: # 顺序采样
            rand = random.randint(0, len(self.buffer) - batch_size)
            batch = [self.buffer[i] for i in range(rand, rand + batch_size)]
            return zip(*batch)
        else: # 随机采样
            batch = random.sample(self.buffer, batch_size)
            return zip(*batch)
        
    def clear(self):
        '''清空经验回放
        '''
        self.buffer.clear()
    
    def __len__(self):
        return len(self.buffer)



#### DQN优化算法

基于梯度下降来实现网络的优化：
$$
\theta_i \leftarrow \theta_i-\lambda \nabla_{\theta_i} L_i (\theta_i)
$$
其中$\theta$就是神经网络的参数。对于损失函数的实现，在DQN中损失设计比较简单：
$$
L(\theta)=(y_i-Q(s_i,a_i;\theta))^2
$$
这里的$y_i$是目标值，$Q(s_i,a_i;\theta)$是当前状态下的Q值。设个损失在深度学习汇总通常称为均方误差损失mesloss。$y_i$在DQN中一般表示如下：
$$
y_{i}= \begin{cases}r_{i} & \text {对于终止状态} s_{i+1} \\ r_{i}+\gamma \max _{a^{\prime}} Q\left(s_{i+1}, a^{\prime} ; \theta\right) & \text {对于非终止状态} s_{i+1}\end{cases}
$$
该公式的意思就是将下一个状态对应的最大Q值作为实际值（因为实际值通常不能直接求得，智能近似）这种做法实际上是一种近似，可能会导致过估计等问题。也有一些改善方法具体可在后面的改进DQN算法中给出。

In [81]:
class DQN:
    def __init__(self, model, memory, cfg):
        self.n_states = cfg.n_states
        self.n_actions = cfg.n_actions
        self.device = torch.device(cfg.device)
        self.gamma = cfg.gamma # 奖励的折扣因子

        # e-greedy策略相关参数
        self.sample_count = 0 # 用于epsilon的衰减计数
        self.epsilon = cfg.epsilon_start
        self.epsilon_start = cfg.epsilon_start
        self.epsilon_end = cfg.epsilon_end
        self.epsilon_decay = cfg.epsilon_decay
        self.batch_size = cfg.batch_size
        self.policy_net = model.to(self.device)
        if cfg.if_load_ckpt:
            self.policy_net.load_state_dict(torch.load(cfg.ckpt_dir + "/DQN_eps20.pth"))
            print("模型加载成功")
        self.target_net = model.to(self.device)

        # 复制参数道目标网络
        for target_param, param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            target_param.data.copy_(param.data)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=cfg.lr)
        self.memory = memory

    def sample_action(self, state):
        '''采样动作
        使用贪婪策略根据当前状态采样动作
        '''
        self.sample_count += 1
        # epsilon-greedy策略
        self.epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
            math.exp(-1. * self.sample_count / self.epsilon_decay)
        if random.random() > self.epsilon:
            # 以概率(1-self.epsilon)选择最优动作 开始很保守 后来大胆探索
            with torch.no_grad():
                state = torch.tensor(state, device=self.device, dtype=torch.float32).unsqueeze(0)
                q_values = self.policy_net(state)
                action = q_values.max(1)[1].item() # 最大值对应的索引 也就是动作值
        else:
            # 以概率self.epsilon 随机选择动作(探索)
            action = random.randrange(self.n_actions)
        return action
    
    @torch.no_grad()
    def predict_action(self, state):
        '''预测动作
        '''
        state = torch.tensor(state, device=self.device, dtype=torch.float32).unsqueeze(0)
        q_values = self.policy_net(state)
        action = q_values.max(1)[1].item() # 最大值对应的索引 也就是动作值
        return action
    
    def update(self,):
        if len(self.memory) < self.batch_size: # 当经验池中的数据量小于batch_size时，不更新参数
            return
        # 从经验池中随机采样batch_size条数据
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.memory.sample(self.batch_size)
        # 将数据转换为tensor
        state_batch = torch.tensor(np.array(state_batch), device=self.device, dtype=torch.float)
        action_batch = torch.tensor(action_batch, device=self.device).unsqueeze(1)
        reward_batch = torch.tensor(reward_batch, device=self.device, dtype=torch.float)
        next_state_batch = torch.tensor(np.array(next_state_batch), device=self.device, dtype=torch.float)
        done_batch = torch.tensor(np.float32(done_batch), device=self.device)

        q_values = self.policy_net(state_batch).gather(1, index=action_batch) # 获取q值 取一个维度上的q值
        next_q_values = self.target_net(next_state_batch).max(1)[0].detach() # 获取下一个状态的q值

        # 计算期望的Q值 对于终止状态 此时done_batch[i]=1 对应的expected_q_values[i] = reward_batch[i]
        expected_q_values = reward_batch + self.gamma * next_q_values * (1 - done_batch)
        loss = nn.MSELoss()(q_values, expected_q_values.unsqueeze(1)) # 计算均方根损失

        # 优化更新模型
        self.optimizer.zero_grad()
        loss.backward()
        # clip防止梯度爆炸
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()


#### 定义训练

In [82]:
def train(cfg, env, agent):
    '''训练
    '''
    print("开始训练！")
    print(f"环境：{cfg.env_name}, 算法：{cfg.algo_name}, 设备：{cfg.device}")
    rewards = []
    steps = []
    for i_ep in range(cfg.train_eps):
        ep_reward = 0 # 记录一轮回合内的奖励
        ep_step = 0 # 记录每一轮的交互次数
        state = env.reset() # 重置环境 返回初始状态
        for _ in range(cfg.ep_max_steps):
            ep_step += 1
            action = agent.sample_action(state) # 选择动作
            next_state, reward, done, _ = env.step(action)
            agent.memory.push((state, action, reward, next_state, done)) # 保存transition到经验池中
            state = next_state
            agent.update()
            ep_reward += reward
            if done:
                break
        if (i_ep+1) % cfg.target_update == 0: # 智能体目标网络更新
            agent.target_net.load_state_dict(agent.policy_net.state_dict())
            #print("更新目标网络.")
        steps.append(ep_step)
        rewards.append(ep_reward)
        if (i_ep+1) % 10 == 0:
            print(f"回合: {i_ep+1}/{cfg.train_eps}, 奖励: {ep_reward:.2f}, 步数: {ep_step}, Epsilon: {agent.epsilon:.2f}")
    print("训练完成!")
    env.close()
    return {"rewards": rewards}

def test(cfg, env, agent):
    print("开始测试！")
    print(f"测试环境: {cfg.env_name}, 算法: {cfg.algo_name}, 设备: {cfg.device}")
    rewards = []
    steps = []
    for i_ep in range(cfg.test_eps):
        ep_reward = 0
        ep_step = 0
        state = env.reset()
        for _ in range(cfg.ep_max_steps):
            ep_step += 1
            action = agent.predict_action(state)
            next_state, reward, done, _ = env.step(action)
            state = next_state
            ep_reward += reward
            if done:
                break
        steps.append(ep_step)
        rewards.append(ep_reward)
        print(f"回合: {i_ep+1}/{cfg.test_eps}, 奖励: {ep_reward:.2f}")
    print("完成测试！")
    env.close()
    return {"rewards": rewards}


#### 定义环境


In [83]:
def all_seed(env, seed = 1):
    ''' 万能的seed函数
    '''
    env.seed(seed) # env config
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed) # config for CPU
    torch.cuda.manual_seed(seed) # config for GPU
    os.environ['PYTHONHASHSEED'] = str(seed) # config for python scripts
    # config for cudnn
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

def env_agent_config(cfg):
    env = gym.make(cfg.env_name)
    if cfg.seed != 0:
        all_seed(env, cfg.seed)
    n_states = env.observation_space.shape[0]
    n_actions = env.action_space.n
    print(f"状态空间维度：{n_states}，动作空间维度：{n_actions}")
    cfg.n_states = n_states
    cfg.n_actions = n_actions
    if torch.cuda.is_available():
        cfg.device = torch.device("cuda")
        print("使用GPU")
    else:
        cfg.device = torch.device("cpu")
        print("使用CPU")
    agent = DQN(model=QNet(n_states=cfg.n_states, n_actions=cfg.n_actions),
                memory=ReplayBuffer(capacity=cfg.memory_capacity),
                cfg=cfg)
    return env, agent

#### 定义参数

In [ ]:
def get_args():
    """配置参数
    """
    parser = argparse.ArgumentParser(description="hyperpaparameter")
    parser.add_argument("--algo_name", default="DQN", type=str, help="算法名称")
    parser.add_argument("--env_name", default="CartPole-v0", type=str, help="环境名称")
    parser.add_argument("--n_states", default=4, type=int, help="状态维度")
    parser.add_argument("--n_actions", default=2, type=int, help="动作维度")
    parser.add_argument("--train_eps", default=200, type=int, help="训练回合数")
    parser.add_argument("--test_eps", default=20, type=int, help="测试回合数")
    parser.add_argument("--ep_max_steps", default=100000, type=int, help="每个回合的最大步数")

    parser.add_argument("--if_load_ckpt", default=False, type=bool, help="是否加载模型")
    parser.add_argument("--ckpt_dir", default="ckpt/", type=str, help="模型存储路径")

    parser.add_argument("--gamma", default=0.95, type=float, help="折扣因子")
    parser.add_argument("--epsilon_start", default=0.95, type=float, help="epsilon的初始值")
    parser.add_argument("--epsilon_end", default=0.01, type=float, help="epsilon的终止值")
    parser.add_argument("--epsilon_decay", default=500, type=int, help="epsilon的衰减值")
    parser.add_argument("--lr", default=0.0001, type=float, help="学习率")

    parser.add_argument("--batch_size", default=64, type=int, help="batch_size")
    parser.add_argument("--target_update", default=4, type=int, help="目标网络更新频率")
    parser.add_argument("--hidden_dim", default=256, type=int, help="隐藏层维度")
    parser.add_argument("--memory_capacity", default=100000, type=int, help="经验池容量")
    parser.add_argument("--device", default="cpu", type=str, help="cpu或者gpu")
    parser.add_argument("--seed", default=10, type=int, help="随机种子")

    _args = parser.parse_args([])
    args = {**vars(_args)} # 将args转换为字典
    # 打印超参数
    print("Hyperparameters:")
    print("".join(['=']*80))
    tplt = "{:^20}\t{:^20}\t{:^20}"
    print(tplt.format("Name", "value", "Type"))
    for k,v in args.items():
        print(tplt.format(k, v, str(type(v))))
    print("".join(["="]*80))
    return _args

def smooth(data, weight=0.9):  
    '''用于平滑曲线，类似于Tensorboard中的smooth曲线
    '''
    last = data[0] 
    smoothed = []
    for point in data:
        smoothed_val = last * weight + (1 - weight) * point  # 计算平滑值
        smoothed.append(smoothed_val)                    
        last = smoothed_val                                
    return smoothed

def plot_rewards(rewards,cfg, tag='train'):
    ''' 画图
    '''
    sns.set()
    plt.figure()  # 创建一个图形实例，方便同时多画几个图
    plt.title(f"{tag}ing curve on {cfg.device} of {cfg.algo_name} for {cfg.env_name}")
    plt.xlabel('epsiodes')
    plt.plot(rewards, label='rewards')
    plt.plot(smooth(rewards), label='smoothed')
    plt.legend()
    plt.show()



#### 开始训练

In [85]:
# 获取参数
cfg = get_args()
env, agent = env_agent_config(cfg)

# 训练
res_dic = train(cfg, env, agent)
plot_rewards(res_dic["rewards"], cfg, tag="train")

# 测试
res_dic = test(cfg, env, agent)
plot_rewards(res_dic["rewards"], cfg, tag="test")

Hyperparameters:
        Name        	       value        	        Type        
     algo_name      	        DQN         	   <class 'str'>    
      env_name      	    CartPole-v0     	   <class 'str'>    
      n_states      	         4          	   <class 'int'>    
     n_actions      	         2          	   <class 'int'>    
     train_eps      	        1000        	   <class 'int'>    
      test_eps      	         20         	   <class 'int'>    
    ep_max_steps    	       100000       	   <class 'int'>    
    if_load_ckpt    	         0          	   <class 'bool'>   
      ckpt_dir      	       ckpt/        	   <class 'str'>    
       gamma        	        0.95        	  <class 'float'>   
   epsilon_start    	        0.95        	  <class 'float'>   
    epsilon_end     	        0.01        	  <class 'float'>   
   epsilon_decay    	        500         	   <class 'int'>    
         lr         	       0.0001       	  <class 'float'>   
     batch_size     	         64      

/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/envs/registration.py:592: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/damonchang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/gym/core.py:268: DeprecationWar

回合: 10/1000, 奖励: 12.00, 步数: 12, Epsilon: 0.65
回合: 20/1000, 奖励: 9.00, 步数: 9, Epsilon: 0.48
回合: 30/1000, 奖励: 15.00, 步数: 15, Epsilon: 0.38
回合: 40/1000, 奖励: 11.00, 步数: 11, Epsilon: 0.31
回合: 50/1000, 奖励: 12.00, 步数: 12, Epsilon: 0.25
回合: 60/1000, 奖励: 40.00, 步数: 40, Epsilon: 0.14
回合: 70/1000, 奖励: 74.00, 步数: 74, Epsilon: 0.04
回合: 80/1000, 奖励: 200.00, 步数: 200, Epsilon: 0.01
回合: 90/1000, 奖励: 200.00, 步数: 200, Epsilon: 0.01
回合: 100/1000, 奖励: 200.00, 步数: 200, Epsilon: 0.01
回合: 110/1000, 奖励: 200.00, 步数: 200, Epsilon: 0.01
回合: 120/1000, 奖励: 200.00, 步数: 200, Epsilon: 0.01


KeyboardInterrupt: 

In [ ]:
cfg

Namespace(algo_name='DQN', env_name='CartPole-v0', n_states=4, n_actions=2, train_eps=1000, test_eps=20, ep_max_steps=100000, if_load_ckpt=False, ckpt_dir='ckpt/', gamma=0.95, epsilon_start=0.95, epsilon_end=0.01, epsilon_decay=500, lr=0.0001, batch_size=64, target_update=4, hidden_dim=256, memory_capacity=100000, device=device(type='cpu'), seed=10)